In [1]:
import sqlite3
import pprint as pp
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
import datetime

In [2]:
def Timer():
    return datetime.datetime.now()

In [3]:
def createTable(filename, tablename):
    createConn = sqlite3.connect(filename)
    createTableQuery = """ CREATE TABLE IF NOT EXISTS %s (
                                        target text,
                                        similar text
                                    ); """ %(tablename)

    createConn.execute(createTableQuery)
    return createConn

In [4]:
def checkTable(createConn, tablename):
    res =  createConn.execute("Select count(*) from %s;" %(tablename))
    print res.fetchall()

def deleteTable(createConn, tablename):
    createConn.execute("drop table %s;"%(tablename))

In [5]:
def insertTable(createConn, tablename, indices, knn):
    counter = 0
    for currentIndex in indices:
        counter = counter+1
        if counter%10000 == 0:
            print "Inserted %d items" %(counter)
        for index in range(1, len(currentIndex)):
            if index == knn:
                break
    #         print artists[currentIndex[0]][0], artists[currentIndex[index]][0]
            insertQuery = """ INSERT INTO %s VALUES ('%s', '%s'); """ %(tablename, ds_song_loudness_tempo[currentIndex[0]][0], ds_song_loudness_tempo[currentIndex[index]][0])
    #         print insertQuery
            createConn.execute(insertQuery)

In [6]:
con2tracks = sqlite3.connect("data/song_feature_db.db")
select_query = "SELECT artist_id,avg(loudness) loud,avg(tempo) as tempo, avg(artist_familiarity) as artist_familiarity,avg(artist_hotttnesss) as artist_hotttnesss,avg(energy) as energy FROM songs GROUP BY artist_id"
song_loudness_tempo = con2tracks.execute(select_query)
ds_song_loudness_tempo = np.array(song_loudness_tempo.fetchall())
print ds_song_loudness_tempo
con2tracks.close()

[[u'AR009211187B989185' u'-9.197' u'149.7485' u'0.39681784431'
  u'0.297404802871' u'0.0']
 [u'AR00A6H1187FB5402A' u'-6.27225' u'155.58725' u'0.514064276419'
  u'0.395627514317' u'0.0']
 [u'AR00LNI1187FB444A5' u'-13.576' u'90.6795' u'0.471212607351'
  u'0.340275507301' u'0.0']
 ..., 
 [u'ARZYP6O1187B9892E7' u'-4.733' u'78.612' u'0.429260855806'
  u'0.339046750241' u'0.0']
 [u'ARZYPLF1187FB45B9B' u'-2.812' u'121.216' u'0.775494507588'
  u'0.542601870216' u'0.0']
 [u'ARZZRK91187B9A5CA5' u'-9.28216666667' u'150.253333333' u'0.58344302355'
  u'0.385787614584' u'0.0']]


In [7]:
ds_lt = ds_song_loudness_tempo[:,1:]
scaler = MinMaxScaler()
scaled_ds_lt = scaler.fit_transform(ds_lt)
print scaled_ds_lt

[[ 0.74781733  0.57890149  0.70194018  0.2747382   0.        ]
 [ 0.82336493  0.60147307  0.76085981  0.3654749   0.        ]
 [ 0.63470579  0.35055107  0.73932564  0.31434153  0.        ]
 ..., 
 [ 0.86312445  0.30390023  0.71824371  0.31320642  0.        ]
 [ 0.91274474  0.46859984  0.89223584  0.50124766  0.        ]
 [ 0.74561743  0.58085308  0.79572458  0.35638495  0.        ]]


/Users/saurabhghotane/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype <U18 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [8]:
start = Timer()
nbrs = NearestNeighbors(n_neighbors=200, algorithm='brute', metric='euclidean').fit(scaled_ds_lt)
distances, indices = nbrs.kneighbors(scaled_ds_lt)
print "Indices = ", indices
print "Distances = ", distances
end = Timer()
print end - start

Indices =  [[   0 1688 2844 ...,   91 1589  342]
 [   1 2802  824 ..., 3490 3336 2801]
 [   2 2862  513 ..., 1260 3322   50]
 ..., 
 [3885  238 3012 ..., 3816 3204  224]
 [3886 1114 2025 ...,  627 3418 3393]
 [3887 3048 3458 ..., 3382 2589 1059]]
Distances =  [[ 0.          0.02344579  0.02753426 ...,  0.09904025  0.09909632
   0.09934167]
 [ 0.          0.02357195  0.0252694  ...,  0.0902396   0.09027295
   0.09043973]
 [ 0.          0.02045428  0.02386873 ...,  0.09605771  0.0960785
   0.09611261]
 ..., 
 [ 0.          0.04164826  0.04298376 ...,  0.13655842  0.13671892
   0.13695353]
 [ 0.          0.04672709  0.04789667 ...,  0.13587588  0.13625159
   0.13625257]
 [ 0.          0.00982516  0.0244393  ...,  0.08242892  0.08251589
   0.08281179]]
0:00:00.531727


## Calculating Accuracy

In [9]:
start = Timer()
# deleteTable(createConn, 'similarity_loudness_tempo_balltree_60')
algo_tbl_name = 'similarity_multifeature_euclidean_60'
createConn = createTable('data/subset_artist_similarity.db',algo_tbl_name)
insertTable(createConn,algo_tbl_name, indices, 60)
checkTable(createConn,algo_tbl_name)
end = Timer()
print end - start

[(229392,)]
0:00:03.455439


In [10]:
start = Timer()
learning_factor=4.1
total = 0.00
for i in range(0,20):
    res = createConn.execute("SELECT * FROM similarity as actual INNER JOIN %s as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(algo_tbl_name, ds_song_loudness_tempo[i][0]))
    final = res.fetchall()
    total = total + len(final)
    end = Timer()
    # print len(final)
print end - start

0:00:15.049446


In [11]:
res = createConn.execute("SELECT count(*) FROM similarity GROUP BY target limit 20;")
counts = res.fetchall()
denominator=0.00
for i in counts:
    denominator = denominator + int(i[0]/learning_factor)

In [12]:
print "Accuracy for Euclidean for 60 Neighbours", total/denominator
createConn.close()

Accuracy for Euclidean for 60 Neighbours 0.535714285714


## Euclidean - 45

In [13]:
start = Timer()
# deleteTable(createConn, 'similarity_loudness_tempo_balltree_60')
algo_tbl_name = 'similarity_multifeature_euclidean_45'
createConn = createTable('data/subset_artist_similarity.db',algo_tbl_name)
insertTable(createConn,algo_tbl_name, indices, 45)
checkTable(createConn,algo_tbl_name)
end = Timer()
print end - start

[(171072,)]
0:00:02.625026


In [14]:
start = Timer()
total = 0.00
for i in range(0,20):
    res = createConn.execute("SELECT * FROM similarity as actual INNER JOIN %s as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(algo_tbl_name, ds_song_loudness_tempo[i][0]))
    final = res.fetchall()
    total = total + len(final)
    end = Timer()
    # print len(final)
print end - start

0:00:11.056617


In [15]:
res = createConn.execute("SELECT count(*) FROM similarity GROUP BY target limit 20;")
counts = res.fetchall()
denominator=0.00
for i in counts:
    denominator = denominator + int(i[0]/learning_factor)

In [16]:
print "Accuracy for Euclidean for 45 Neighbours", total/denominator
createConn.close()

Accuracy for Euclidean for 45 Neighbours 0.446428571429


## Euclidean - 90

In [21]:
start = Timer()
# deleteTable(createConn, 'similarity_loudness_tempo_balltree_60')
algo_tbl_name = 'similarity_multifeature_euclidean_90'
createConn = createTable('data/subset_artist_similarity.db',algo_tbl_name)
insertTable(createConn,algo_tbl_name, indices, 90)
checkTable(createConn,algo_tbl_name)
end = Timer()
print end - start

[(346032,)]
0:00:05.989694


In [22]:
start = Timer()
learning_factor=2.7
total = 0.00
for i in range(0,20):
    res = createConn.execute("SELECT * FROM similarity as actual INNER JOIN %s as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(algo_tbl_name, ds_song_loudness_tempo[i][0]))
    final = res.fetchall()
    total = total + len(final)
    end = Timer()
    # print len(final)
print end - start

0:00:22.927289


In [23]:
res = createConn.execute("SELECT count(*) FROM similarity GROUP BY target limit 20;")
counts = res.fetchall()
denominator=0.00
for i in counts:
    denominator = denominator + int(i[0]/learning_factor)

In [24]:
print "Accuracy for Euclidean for 90 Neighbours", total/denominator
createConn.close()

Accuracy for Euclidean for 90 Neighbours 0.404255319149
